In [9]:
import pandas as pd

# Đọc file Excel
df = pd.read_excel('Chatbot_Custom.xlsx', engine='openpyxl')

# Hiển thị dữ liệu
print(df.head())

         title                                Question  Answer_label  \
0  1719/QĐ-TTg             Mục tiêu của dự án 2 là gì?             0   
1  1719/QĐ-TTg              Dự án 2 có mục tiêu là gì?             0   
2  1719/QĐ-TTg                   Mục tiêu của dự án 2?             0   
3  1719/QĐ-TTg  Đối tượng thụ hưởng của dự án 2 là ai?             1   
4  1719/QĐ-TTg        Đối tượng thụ hưởng của dự án 2?             1   

                                              Answer  
0  Mục tiêu của Dự án 2 thuộc Chương trình mục ti...  
1  Mục tiêu của Dự án 2 thuộc Chương trình mục ti...  
2  Mục tiêu của Dự án 2 thuộc Chương trình mục ti...  
3  Đối tượng thụ hưởng của Dự án 2 thuộc Chương t...  
4  Đối tượng thụ hưởng của Dự án 2 thuộc Chương t...  


In [10]:
# Đảm bảo cột 'label' là số nguyên
df['Answer_label'] = df['Answer_label'].astype(int)

# Chọn một hàng ngẫu nhiên từ mỗi nhóm
eval_df = df.groupby('Answer_label').apply(lambda x: x.sample(n=1)).reset_index(drop=True)

# Hiển thị DataFrame mới
print(eval_df.head())


         title                          Question  Answer_label  \
0  1719/QĐ-TTg       Mục tiêu của dự án 2 là gì?             0   
1  1719/QĐ-TTg  Đối tượng thụ hưởng của dự án 2?             1   
2  1719/QĐ-TTg        Dự án 2 có nội dung là gì?             2   
3  1719/QĐ-TTg           Nội dung chính dự án 2?             3   
4  1719/QĐ-TTg    Số nội dung chính của dự án 2?             4   

                                              Answer  
0  Mục tiêu của Dự án 2 thuộc Chương trình mục ti...  
1  Đối tượng thụ hưởng của Dự án 2 thuộc Chương t...  
2  Nội dung của Dự án 2 thuộc Chương trình mục ti...  
3  Nội dung chính của Dự án 2 thuộc Chương trình ...  
4  Dự án 2 thuộc Chương trình mục tiêu quốc gia p...  


C:\Users\Admin\AppData\Local\Temp\ipykernel_16284\620973280.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  eval_df = df.groupby('Answer_label').apply(lambda x: x.sample(n=1)).reset_index(drop=True)


In [11]:
# Lấy các giá trị cột 'question' từ eval_df
questions_in_eval_df = eval_df['Question'].tolist()

# Lọc df để loại bỏ các hàng có giá trị cột 'question' có trong eval_df
train_df = df[~df['Question'].isin(questions_in_eval_df)]

# Hiển thị DataFrame đã lọc
print(train_df.head())

         title                                    Question  Answer_label  \
1  1719/QĐ-TTg                  Dự án 2 có mục tiêu là gì?             0   
2  1719/QĐ-TTg                       Mục tiêu của dự án 2?             0   
3  1719/QĐ-TTg      Đối tượng thụ hưởng của dự án 2 là ai?             1   
5  1719/QĐ-TTg  Dự án 2 hướng tới đối tượng thụ hưởng nào?             1   
6  1719/QĐ-TTg                 Nội dung của dự án 2 là gì?             2   

                                              Answer  
1  Mục tiêu của Dự án 2 thuộc Chương trình mục ti...  
2  Mục tiêu của Dự án 2 thuộc Chương trình mục ti...  
3  Đối tượng thụ hưởng của Dự án 2 thuộc Chương t...  
5  Đối tượng thụ hưởng của Dự án 2 thuộc Chương t...  
6  Nội dung của Dự án 2 thuộc Chương trình mục ti...  


In [12]:
train_df1 = train_df[['Question', 'Answer']]
eval_df1 = eval_df[['Question', 'Answer']]

In [13]:
# Chuyển đổi dữ liệu DataFrame thành định dạng JSON để phù hợp với Dataset.from_list
train_data = train_df1.to_dict(orient='records')
eval_data = eval_df1.to_dict(orient='records')

In [2]:
import torch

In [3]:
torch.cuda.empty_cache()

In [16]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from bitsandbytes.optim import Adam8bit
from datasets import Dataset, DatasetDict

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-syllable")
model = AutoModelForSeq2SeqLM.from_pretrained("vinai/bartpho-syllable")

# Enable gradient checkpointing
model.gradient_checkpointing_enable()

def preprocess_function(examples):
    inputs = tokenizer(examples['Question'], max_length=512, truncation=True, padding='max_length')
    targets = tokenizer(examples['Answer'], max_length=512, truncation=True, padding='max_length')
    inputs['labels'] = targets['input_ids']
    return inputs

# Convert DataFrame to Dataset
train_dataset = Dataset.from_list(train_data)
eval_dataset = Dataset.from_list(eval_data)

# Tokenize dataset
tokenized_datasets = DatasetDict({
    'train': train_dataset.map(preprocess_function, batched=True),
    'eval': eval_dataset.map(preprocess_function, batched=True)
})

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    num_train_epochs=10,
    logging_dir='./logs',
    logging_steps=2,
    eval_strategy="epoch",
    output_dir='./tests',
    fp16=True
)

# Use Adam8bit optimizer
optim = Adam8bit(model.parameters(), lr=5e-5)

# Create the Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['eval'],
    optimizers=(optim, None)  # Use Adam8bit optimizer
)

# Clear cache before starting training
torch.cuda.empty_cache()

# Fine-tune the model
trainer.train()

C:\Users\Admin\PycharmProjects\pythonProject\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 426/426 [00:00<00:00, 3407.09 examples/s]
C:\Users\Admin\PycharmProjects\pythonProject\.venv\Lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


AssertionError: Torch not compiled with CUDA enabled

In [18]:
import torch
print(torch.__version__)          # Kiểm tra phiên bản PyTorch


False
2.4.1+cpu
